# Digit Recognizier

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In the train data the first column is label and each image is 28*28 pixel by pixel image. Then from 2nd column to last column that is 784 columns represent the pixel values of each image. There are totally 42000 images in the train data set.

## My plan is to take the train data and use pca to reduce the dimensions and then use logistic regression to classify the images

In [27]:
X=np.array(train)
(m,n)=X.shape
y=X[:,0:1]
X=X[:,1:]/255           # normalising the data ( increase the speed of computation (this is just the observation))
n=n-1
print(X.shape,y.shape)

(42000, 784) (42000, 1)


In [28]:
# I have removed all the columns whose varaince is zero....
from sklearn.feature_selection  import VarianceThreshold
vr=VarianceThreshold()
vr.fit(X)
X=vr.transform(X)
print(X.shape)

(42000, 708)


In [7]:
X_train,X_cross,y_train,y_cross=train_test_split(X,y,test_size=0.2)

In [8]:
print(X_train.shape,y_train.shape,y_cross.shape,X_cross.shape)

(33600, 708) (33600, 1) (8400, 1) (8400, 708)


In [9]:
pca=PCA(n_components=200)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [10]:
x=pca.explained_variance_ratio_.cumsum()               # will give a vector where x[i] is variance of i+1 dimension
print(x[149],x[199])

0.948698241141 0.966334909225


## PCA with 200 components 

Important Note: In implementation of PCA we actually take the first k columns of the matrix u.
In sklearn library what it does is it gives U transpose that is it takes first k columns and make it to first k rows and give the matrix

Thats why in the below code i have taken ureduce as U.T only due to this reason

In [11]:
u=pca.components_                       
print(u.shape)
ureduce=u.T
print(ureduce.shape)

(200, 708)
(708, 200)


Here i am using pca.transform function that is it project our data into desirable dimensions

In [12]:
X_train_red=pca.transform(X_train)
print(X_train_red.shape)

(33600, 200)


Actually when we are using inbuilt logistic regression our y must have shape of (n_samples,)

In [13]:
y_train=y_train.reshape(y_train.size,)
y_cross=y_cross.reshape(y_cross.size,)
print(y_train.shape,y_cross.shape)

(33600,) (8400,)


Building feature matrix 

In [14]:
X_train_red_f=np.hstack((X_train_red,X_train_red**2))
print(X_train_red_f.shape)

(33600, 400)


## training the data using logistic regression

In [15]:
lr=LogisticRegression(solver='lbfgs',multi_class='multinomial')
lr.fit(X_train_red_f,y_train)

train_score=lr.score(X_train_red_f,y_train)
print(train_score)

0.958035714286


In [17]:
X_cross_red=pca.transform(X_cross)
print(X_cross_red.shape)

(8400, 200)


In [18]:
X_cross_red_f=np.hstack((X_cross_red,X_cross_red**2))
print(X_cross_red_f.shape)
cross_score=lr.score(X_cross_red_f,y_cross)
print(cross_score);

(8400, 400)
0.935833333333


In [29]:
X_test=np.array(test)/255
X_test=vr.transform(X_test)
print(X_test.shape)

(28000, 708)


In [30]:
X_test_red=pca.transform(X_test)
print(X_test_red.shape)

(28000, 200)


In [31]:
X_test_red_f=np.hstack((X_test_red,X_test_red**2))
print(X_test_red_f.shape)

y_test=lr.predict(X_test_red_f)
y_test.shape=(28000,1)

imageid=np.linspace(1,28000,28000).reshape(28000,1)
ans=np.hstack((imageid,y_test))
print(ans.shape)

ans=pd.DataFrame(ans,columns=['ImageId','Label'])
ans=ans.astype(int)
ans.head(10)

(28000, 400)
(28000, 2)


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [32]:
  ans.to_csv("pred_pca_200_one.csv",index=False)               # got 93.657% accuracy